In [1]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import nltk
nltk.download("all")
import matplotlib.pyplot as plt
import torch

%matplotlib inline

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

In [2]:
from nltk.tokenize import word_tokenize
from collections import defaultdict
from tqdm import tqdm_notebook #notebook.tqdm
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data import RandomSampler
from torch.utils.data import SequentialSampler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import time

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
def load_text(path):
  file = open(path,'rb')
  text = []
  for line in file:
    text.append(line.decode(errors='ignore').lower().strip())
  return text

very_neg_text = load_text('/content/drive/My Drive/data/train0')
neg_text = load_text('/content/drive/My Drive/data/train1')
neu_text = load_text('/content/drive/My Drive/data/train2')
pos_text = load_text('/content/drive/My Drive/data/train3')
very_pos_text = load_text('/content/drive/My Drive/data/train4')




texts = np.array(very_neg_text + neg_text+ neu_text +pos_text + very_pos_text)
labels = np.array([0]*len(very_neg_text) + [1]*len(neg_text) + [2]*len(neu_text) +[3]*len(pos_text) +[4]* len(very_pos_text))

In [5]:
print(len(texts))

9645


In [6]:
URL = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip"
FILE = "fastText"

if os.path.isdir(FILE):
    print("fastText exists.")
else:
    !wget -P $FILE $URL
    !unzip $FILE/crawl-300d-2M.vec.zip -d $FILE

--2021-05-19 17:46:55--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1523785255 (1.4G) [application/zip]
Saving to: ‘fastText/crawl-300d-2M.vec.zip’

crawl-300d-2M.vec.z 100%[===================>]   1.42G  12.2MB/s    in 2m 1s   

2021-05-19 17:48:57 (12.0 MB/s) - ‘fastText/crawl-300d-2M.vec.zip’ saved [1523785255/1523785255]

Archive:  fastText/crawl-300d-2M.vec.zip
  inflating: fastText/crawl-300d-2M.vec  


In [7]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("No. of GPU(s) available: {}".format(torch.cuda.device_count()))
    print('Device name:', torch.cuda.get_device_name(0))       
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No. of GPU(s) available: 1
Device name: Tesla K80


In [24]:
def tokenize(texts):

    max_len = 0
    tokenized_texts = []
    word2idx = {}
    lengths_of_sentences = []

    # <pad> refers to padding and <unk> refers to unknown word 
    word2idx['<pad>'] = 0
    word2idx['<unk>'] = 1

    idx = 2
    for sent in texts:
        tokenized_sent = word_tokenize(sent)
        tokenized_texts.append(tokenized_sent)
        count = 0
        
        for token in tokenized_sent:
            count+=1
            if token not in word2idx:
                word2idx[token] = idx
                idx += 1
        lengths_of_sentences.append(count)
                
        max_len = max(max_len, len(tokenized_sent))
    lengths_of_sentences = np.array(lengths_of_sentences)
    print("Average Sentence Length = {}".format(np.mean(lengths_of_sentences)))

    return tokenized_texts, word2idx, max_len

In [25]:
def encode(tokenized_texts, word2idx, max_len):
    input_ids = []
    for tokenized_sent in tokenized_texts:
        # Padding the sentences to max_len
        tokenized_sent += ['<pad>'] * (max_len - len(tokenized_sent))

        # Encode tokens to input_ids
        input_id = [word2idx.get(token) for token in tokenized_sent]
        input_ids.append(input_id)
    
    return np.array(input_ids)

In [26]:
def load_pretrained_vectors(word2idx, fname):
    print("Loading pretrained vectors...")
    fin = open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())

    # Initilize random embeddings
    embeddings = np.random.uniform(-0.25, 0.25, (len(word2idx), d))
    embeddings[word2idx['<pad>']] = np.zeros((d,))

    # Load pretrained vectors
    count = 0
    for line in tqdm_notebook(fin):
        tokens = line.rstrip().split(' ')
        word = tokens[0]
        if word in word2idx:
            count += 1
            embeddings[word2idx[word]] = np.array(tokens[1:], dtype=np.float32)

    print("There are {} / {} pretrained vectors found.".format(count,len(word2idx)))

    return embeddings


In [27]:
# Tokenize, build vocabulary, encode tokens
print("Tokenizing...\n")
tokenized_texts, word2idx, max_len = tokenize(texts)
input_ids = encode(tokenized_texts, word2idx, max_len)

# Load pretrained vectors
embeddings = load_pretrained_vectors(word2idx, "fastText/crawl-300d-2M.vec")
embeddings = torch.tensor(embeddings)

Tokenizing...

Average Sentence Length = 19.171798859512702
Loading pretrained vectors...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':



There are 16405 / 17604 pretrained vectors found.


In [28]:
def data_loader(train_inputs, val_inputs, train_labels, val_labels,batch_size=50):

    train_inputs = torch.tensor(train_inputs)
    train_labels = torch.tensor(train_labels)
    val_inputs = torch.tensor(val_inputs)
    val_labels = torch.tensor(val_labels)

    # Create DataLoader for training data
    train_data = TensorDataset(train_inputs, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=50)

    # Create DataLoader for validation data
    val_data = TensorDataset(val_inputs, val_labels)
    val_sampler = SequentialSampler(val_data)
    val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

    return train_dataloader, val_dataloader

In [29]:


# Train Test Split
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, labels, test_size=0.1, random_state=42)

# Load data to PyTorch DataLoader
train_dataloader, val_dataloader = \
data_loader(train_inputs, val_inputs, train_labels, val_labels, batch_size=50)

In [30]:
class CNN_NLP(nn.Module):
    def __init__(self,
                 pretrained_embedding=None,
                 freeze_embedding=False,
                 vocab_size=None,
                 embed_dim=300,
                 filter_sizes=[3, 4, 5],
                 num_filters=[100, 100, 100],
                 num_classes=5,
                 dropout=0.5):

        super(CNN_NLP, self).__init__()
        # Embedding layer
        if pretrained_embedding is not None:
            self.vocab_size, self.embed_dim = pretrained_embedding.shape
            self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)
        else:
            self.embed_dim = embed_dim
            self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=self.embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        # Conv Network
        self.conv1d_list = nn.ModuleList([
            nn.Conv1d(in_channels=self.embed_dim,
                      out_channels=num_filters[i],
                      kernel_size=filter_sizes[i])
            for i in range(len(filter_sizes))
        ])
        # Fully-connected layer and Dropout
        self.fc = nn.Linear(np.sum(num_filters), num_classes)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, input_ids):

        x_embed = self.embedding(input_ids).float()
        x_reshaped = x_embed.permute(0, 2, 1)
        #Applying CNN and Relu
        x_conv_list = [F.relu(conv1d(x_reshaped)) for conv1d in self.conv1d_list]

        # Max pooling
        x_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2])
            for x_conv in x_conv_list]
        
        x_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list],
                         dim=1)
        
        logits = self.fc(self.dropout(x_fc))

        return logits

In [31]:
def initilize_model(pretrained_embedding=None,
                    freeze_embedding=False,
                    vocab_size=None,
                    embed_dim=300,
                    filter_sizes=[3, 4, 5],
                    num_filters=[100, 100, 100],
                    num_classes=5,
                    dropout=0.5,
                    learning_rate=0.01):

    assert (len(filter_sizes) == len(num_filters)), "filter_sizes and \
    num_filters need to be of the same length."

    # Instantiate CNN model
    cnn_model = CNN_NLP(pretrained_embedding=pretrained_embedding,
                        freeze_embedding=freeze_embedding,
                        vocab_size=vocab_size,
                        embed_dim=embed_dim,
                        filter_sizes=filter_sizes,
                        num_filters=num_filters,
                        num_classes=5,
                        dropout=0.5)
    
    cnn_model.to(device)

    # Instantiate Adadelta optimizer
    optimizer = optim.Adadelta(cnn_model.parameters(),
                               lr=learning_rate,
                               rho=0.95)

    return cnn_model, optimizer


In [32]:
# Specify loss function
#Here we are using cross entropy loss function
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):

    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, optimizer, train_dataloader, val_dataloader=None, epochs=10):
    
    # Tracking best validation accuracy
    best_accuracy = 0

    # Start training loop
    print("Start training...\n")
    print(f"{'Epoch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {\
    'Val Acc':^9} | {'Elapsed':^9}")
    print("-"*60)

    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================

        # Tracking time and loss
        t0_epoch = time.time()
        total_loss = 0

        # Put the model into the training mode
        model.train()

        for step, batch in enumerate(train_dataloader):
            # Load batch to GPU
            b_input_ids, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Update parameters
            optimizer.step()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        # =======================================
        #               Evaluation
        # =======================================
        if val_dataloader is not None:
            # After the completion of each training epoch, measure the model's
            # performance on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Track the best accuracy
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            print(f"{epoch_i + 1:^7} | {avg_train_loss:^12.6f} | {\
            val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            
    print("\n")
    print(f"Training complete! Best accuracy: {best_accuracy:.2f}%.")

def evaluate(model, val_dataloader):
    # Put the model into the evaluation mode. The dropout layers are disabled
    # during the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

In [33]:
# CNN-rand: Word vectors are randomly initialized.
set_seed(42)
cnn_rand, optimizer = initilize_model(vocab_size=len(word2idx),
                                      embed_dim=300,
                                      learning_rate=0.25,
                                      dropout=0.5)
train(cnn_rand, optimizer, train_dataloader, val_dataloader, epochs=20)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------
   1    |   1.569199   |  1.534224  |   34.00   |   2.88   
   2    |   1.482414   |  1.518440  |   30.43   |   2.44   
   3    |   1.392329   |  1.475154  |   35.97   |   2.45   
   4    |   1.276637   |  1.445801  |   36.87   |   2.44   
   5    |   1.152451   |  1.424360  |   38.00   |   2.44   
   6    |   1.018747   |  1.415150  |   38.50   |   2.44   
   7    |   0.885532   |  1.420037  |   38.53   |   2.44   
   8    |   0.767678   |  1.432974  |   36.73   |   2.45   
   9    |   0.652519   |  1.451346  |   38.27   |   2.45   
  10    |   0.551135   |  1.464990  |   37.70   |   2.45   
  11    |   0.463017   |  1.490497  |   37.50   |   2.44   
  12    |   0.391070   |  1.515859  |   37.23   |   2.44   
  13    |   0.333906   |  1.544521  |   38.47   |   2.44   
  14    |   0.282251   |  1.587139  |   37.83   |   2.45   
  15    |   0.246858

In [34]:
# CNN-static: fastText pretrained word vectors are used and freezed during training.
set_seed(42)
cnn_static, optimizer = initilize_model(pretrained_embedding=embeddings,
                                        freeze_embedding=True,
                                        learning_rate=0.25,
                                        dropout=0.5)
train(cnn_static, optimizer, train_dataloader, val_dataloader, epochs=20)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------
   1    |   1.504497   |  1.373582  |   40.13   |   1.28   
   2    |   1.306942   |  1.287849  |   45.07   |   1.28   
   3    |   1.212227   |  1.260119  |   44.53   |   1.27   
   4    |   1.133150   |  1.247208  |   45.93   |   1.28   
   5    |   1.053660   |  1.225730  |   47.83   |   1.28   
   6    |   0.961244   |  1.237754  |   46.13   |   1.28   
   7    |   0.879807   |  1.246366  |   44.00   |   1.28   
   8    |   0.783889   |  1.251814  |   45.27   |   1.28   
   9    |   0.704604   |  1.238977  |   46.07   |   1.29   
  10    |   0.609638   |  1.280187  |   45.50   |   1.27   
  11    |   0.532898   |  1.310420  |   44.37   |   1.29   
  12    |   0.462984   |  1.305693  |   45.17   |   1.28   
  13    |   0.405089   |  1.312592  |   45.73   |   1.29   
  14    |   0.355407   |  1.339849  |   44.47   |   1.29   
  15    |   0.309270

In [35]:
# CNN-non-static: fastText pretrained word vectors are fine-tuned during training.
set_seed(42)
cnn_non_static, optimizer = initilize_model(pretrained_embedding=embeddings,
                                            freeze_embedding=False,
                                            learning_rate=0.25,
                                            dropout=0.5)
train(cnn_non_static, optimizer, train_dataloader, val_dataloader, epochs=20)


Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------
   1    |   1.504039   |  1.371842  |   40.63   |   3.39   
   2    |   1.302998   |  1.285302  |   44.97   |   3.40   
   3    |   1.203186   |  1.256142  |   44.23   |   3.40   
   4    |   1.118899   |  1.240667  |   45.30   |   3.40   
   5    |   1.033075   |  1.215054  |   47.73   |   3.40   
   6    |   0.936593   |  1.229398  |   45.90   |   3.40   
   7    |   0.845962   |  1.245756  |   44.53   |   3.40   
   8    |   0.745788   |  1.238997  |   46.00   |   3.40   
   9    |   0.659484   |  1.232785  |   46.90   |   3.40   
  10    |   0.562946   |  1.276033  |   45.60   |   3.41   
  11    |   0.485430   |  1.298472  |   44.67   |   3.41   
  12    |   0.415439   |  1.301968  |   46.47   |   3.40   
  13    |   0.354830   |  1.319315  |   45.20   |   3.40   
  14    |   0.300595   |  1.345031  |   46.67   |   3.40   
  15    |   0.264672

In [36]:
def predict(text, model=cnn_non_static.to("cpu"), max_len=62):
    """Predict probability that a review is positive."""

    # Tokenize, pad and encode text
    tokens = word_tokenize(text.lower())
    padded_tokens = tokens + ['<pad>'] * (max_len - len(tokens))
    input_id = [word2idx.get(token, word2idx['<unk>']) for token in padded_tokens]

    # Convert to PyTorch tensors
    input_id = torch.tensor(input_id).unsqueeze(dim=0)

    # Compute logits
    logits = model.forward(input_id)

    #  Compute probability
    probs = F.softmax(logits, dim=1).squeeze(dim=0)
    classes = ['very negative','negative','neutral','positive','very positive']
    print(f"This review is {max(probs) * 100:.2f}% {classes[torch.argmax(probs)]}.")


In [40]:
predict("Perhaps no picture ever made has more literally showed that the road to hell is paved with good intentions .")
predict("A thoughtful , provocative , insistently humanizing film .")
predict("Illuminating if overly talky documentary .")
predict("Watching Haneke 's film is , aptly enough , a challenge and a punishment .")
predict("Worst of all!")


This review is 77.57% positive.
This review is 72.38% very positive.
This review is 83.03% neutral.
This review is 65.61% positive.
This review is 86.50% very negative.
